In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from copy import deepcopy
from collections import defaultdict
import json
import warnings

from joblib import Parallel, delayed
import k3d
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import yaml

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-56o0phfy because the default path (/home/user/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='flat'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = np.max(samples_distances)

            colors = k3d.helpers.map_colors(
                samples_distances, k3d.colormaps.matplotlib_color_maps.hot, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            vectors = k3d.vectors(samples, directions * samples_distances[..., np.newaxis])
            print(vectors)
            plot += vectors

#             directions_to_plot = np.hstack((samples, samples + directions))
            
#             for i, dir_to_plot in enumerate(directions_to_plot):
#                 dir_to_plot = dir_to_plot.reshape((2, 3))
#                 if np.all(dir_to_plot[0] == dir_to_plot[1]):
#                     continue
#                 color = int(colors[i]) if None is not colors else samples_color
#                 plt_line = k3d.line(dir_to_plot, 
#                                     shader='mesh', width=directions_width, color=color)
#                 plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not sharp_curves:            
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                import randomcolor
                rand_color = randomcolor.RandomColor()
            for i, vert_ind in enumerate(sharp_curves):
                sharp_points_curve = mesh.vertices[vert_ind]
                
                if None is sharpcurve_color:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [4]:
from sharpf.utils.abc_utils.hdf5.dataset import Hdf5File, PreloadTypes
import sharpf.utils.abc_utils.hdf5.io_struct as io
from sharpf.data.datasets.sharpf_io import PointCloudIO

In [5]:
PointCloudIO = io.HDF5IO({
    'points': io.Float64('points'),
    'normals': io.Float64('normals'),
    'distances': io.Float64('distances'),
    'directions': io.Float64('directions'),
    'item_id': io.AsciiString('item_id'),
    'orig_vert_indices': io.VarInt32('orig_vert_indices'),
    'orig_face_indexes': io.VarInt32('orig_face_indexes'),
    'has_sharp': io.Bool('has_sharp'),
    'num_sharp_curves': io.Int8('num_sharp_curves'),
    'num_surfaces': io.Int8('num_surfaces'),
    'has_smell_coarse_surfaces_by_num_faces': io.Bool('has_smell_coarse_surfaces_by_num_faces'),
    'has_smell_coarse_surfaces_by_angles': io.Bool('has_smell_coarse_surfaces_by_angles'),
    'has_smell_deviating_resolution': io.Bool('has_smell_deviating_resolution'),
    'has_smell_sharpness_discontinuities': io.Bool('has_smell_sharpness_discontinuities'),
    'has_smell_bad_face_sampling': io.Bool('has_smell_bad_face_sampling'),
    'has_smell_mismatching_surface_annotation': io.Bool('has_smell_mismatching_surface_annotation'),
# 'voronoi': io.Float64('voronoi'),
# 'normals_estimation_10': io.Float64('normals_estimation_10'),
# 'normals_estimation_100': io.Float64('normals_estimation_100'),

},
len_label='has_sharp',
compression='lzf')

In [56]:
dataset = Hdf5File('/logs/abc_0001_1920_1930_0.01.hdf5',
                   io=PointCloudIO,
                   preload=PreloadTypes.NEVER,
                   labels='*')

In [57]:
len(dataset)

439

In [86]:
idx = np.random.randint(0, len(dataset) - 1)

item = dataset[idx]

points = item['points']
directions = item['directions']
distances = item['distances']

display_sharpness(
    mesh=None,
    samples=points, samples_psize=0.02,
    samples_distances=distances,
    directions=None)

print(item)



Output()

{'has_smell_bad_face_sampling': True, 'orig_vert_indices': array([ 765,  766,  767,  768,  769, 4169, 4436, 4452, 4738, 4742, 4921,
       5129, 5374, 6654, 6655, 6729, 6730, 6731, 6797, 6798, 6799, 6800],
      dtype=int32), 'has_smell_coarse_surfaces_by_angles': False, 'has_smell_mismatching_surface_annotation': True, 'points': array([[ 2.55340377e+01,  4.77680139e-02,  2.49342354e-02],
       [ 2.55239501e+01,  4.89991487e-02,  3.20991754e-02],
       [ 2.55630740e+01,  4.52011791e-02,  1.74136884e-02],
       ...,
       [ 2.49090483e+01, -6.80780171e-01,  1.06890820e-02],
       [ 2.54751406e+01,  9.67529922e-01,  5.29815512e-03],
       [ 2.48185696e+01, -5.91132155e-01, -9.37373100e-03]]), 'num_sharp_curves': 1, 'directions': array([[ 0.94595734,  0.06200048, -0.31826893],
       [ 0.93246753,  0.06111669, -0.35601526],
       [ 0.93182011,  0.06107438, -0.35768718],
       ...,
       [ 0.98063863, -0.1950614 , -0.01719624],
       [ 0.87287383,  0.17360714, -0.45582866],
     

In [78]:
from scipy.spatial import cKDTree


In [89]:
nn_distances, nn_indexes = cKDTree(points, leafsize=16).query(points, k=2, n_jobs=2)
values = np.abs(distances[nn_indexes[:, 0]] - distances[nn_indexes[:, 1]]) / nn_distances[:, 1]
# warnings.warn('Discontinuities found in SDF values, discarding patch')

In [84]:
distances[nn_indexes[:, 0]] - distances[nn_indexes[:, 1]]

array([ 0.01224533, -0.00732714,  0.00963871, ...,  0.01755166,
        0.00610714,  0.00504708])

In [83]:
nn_distances[:, 1]

array([0.01401113, 0.0080415 , 0.01373282, ..., 0.04356874, 0.0188934 ,
       0.00779316])

In [ ]:
from sharpf.data import DataGenerationException
from sharpf.utils.abc_utils.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.datasets.sharpf_io import save_point_patches
from sharpf.data.mesh_nbhoods import NBHOOD_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.data.point_samplers import SAMPLER_BY_TYPE
from sharpf.utils.abc_utils.abc.feature_utils import compute_features_nbhood, remove_boundary_features, get_curves_extents
from sharpf.utils.py_utils.console import eprint_t
from sharpf.utils.py_utils.os import add_suffix
from sharpf.utils.py_utils.config import load_func_from_config
from sharpf.utils.abc_utils.mesh.io import trimesh_load
from sharpf.utils.geometry import mean_mmd
import sharpf.data.data_smells as smells

In [ ]:
with ABCChunk(['/data/abc/abc_0001_obj_v00.7z', '/data/abc/abc_0001_feat_v00.7z']) as data_holder:
    item_abc = data_holder.get(item['item_id'].decode())

In [ ]:
# mesh, vertex_normals, vertex_normal_indices = trimesh_load(item.obj)
mesh, _, _ = trimesh_load(item_abc.obj)

features = yaml.load(item_abc.feat, Loader=yaml.Loader)

In [ ]:
with open('/code/scripts/data_scripts/configs/pointcloud_datasets/dataset_config_high_res_clean.json') as config_file:
    config = json.load(config_file)

config

In [ ]:
def scale_mesh(mesh, features, shape_fabrication_extent, resolution_3d,
               short_curve_quantile=0.05, n_points_per_short_curve=4):
    # compute standard size spatial extent
    mesh_extent = np.max(mesh.bounding_box.extents)
    mesh = mesh.apply_scale(shape_fabrication_extent / mesh_extent)

    # compute lengths of curves
    sharp_curves_lengths = get_curves_extents(mesh, features)

    least_len = np.quantile(sharp_curves_lengths, short_curve_quantile)
    least_len_mm = resolution_3d * n_points_per_short_curve

    mesh = mesh.apply_scale(least_len_mm / least_len)

    return mesh

In [ ]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
base_n_points_per_short_curve = config.get('base_n_points_per_short_curve', 8)
base_resolution_3d = config.get('base_resolution_3d', LOW_RES)

short_curve_quantile = config.get('short_curve_quantile', 0.05)

mesh = scale_mesh(mesh, features, shape_fabrication_extent, base_resolution_3d,
                      short_curve_quantile=short_curve_quantile,
                      n_points_per_short_curve=base_n_points_per_short_curve)

In [ ]:
annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])

In [ ]:
# surf_vert_indices = np.array(features['surfaces'][0]['vert_indices'])
# surf_verts = mesh.vertices[surf_vert_indices]
# surf_vert_indices2 = np.array(features['surfaces'][0]['vert_indices'])
# surf_verts2 = mesh.vertices[surf_vert_indices]


display_sharpness(
    mesh, plot_meshvert=True, meshvert_psize=HIGH_RES,
    sharp_vert=None, 
    sharpvert_psize=HIGH_RES,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
    samples_psize=HIGH_RES,
    samples=item['points'],
    samples_distances=distances,
    directions=directions
)

In [ ]:
from sharpf.utils.abc_utils.mesh.indexing import reindex_array, reindex_zerobased
from sharpf.utils.abc_utils.abc.feature_utils import get_surface_as_mesh

In [ ]:
nbhood = reindex_zerobased(mesh, item['orig_vert_indices'], item['orig_face_indexes'])

In [ ]:

def compute_features_nbhood(
        mesh,
        features,
        mesh_face_indexes,
        mesh_vertex_indexes=None,
        deduce_verts_from_faces=False,
):
    """Extracts curves for the neighbourhood."""

    if deduce_verts_from_faces:
        mesh_vertex_indexes = np.unique(mesh.faces[mesh_face_indexes].ravel())

    nbhood_curves = []
    for curve in features['curves']:
        curve_vertex_indexes = np.array(curve['vert_indices'])
        curve_vertex_indexes = curve_vertex_indexes[
            np.where(np.isin(curve_vertex_indexes, mesh_vertex_indexes))[0]]
        if len(curve_vertex_indexes) == 0:
            continue

        curve_vertex_indexes = reindex_array(curve_vertex_indexes, mesh_vertex_indexes)

        nbhood_curve = deepcopy(curve)
        nbhood_curve['vert_indices'] = curve_vertex_indexes
        nbhood_curves.append(nbhood_curve)

    nbhood_surfaces = []
    for idx, surface in enumerate(features['surfaces']):
        surface_face_indexes = np.array(surface['face_indices'])
        surface_face_indexes = surface_face_indexes[
            np.where(np.isin(surface_face_indexes, mesh_face_indexes))[0]]
        if len(surface_face_indexes) == 0:
            continue

        surface_faces = reindex_array(mesh.faces[surface_face_indexes], mesh_vertex_indexes)

        nbhood_surface = deepcopy(surface)
        nbhood_surface['face_indices'] = reindex_array(surface_face_indexes, mesh_face_indexes)
        nbhood_surface['vert_indices'] = np.unique(surface_faces)
        nbhood_surfaces.append(nbhood_surface)

    nbhood_features = {
        'curves': nbhood_curves,
        'surfaces': nbhood_surfaces,
    }
    return nbhood_features


In [ ]:

def get_boundary_vertex_indexes(mesh):
    mesh_edge_indexes, mesh_edge_counts = np.unique(
        mesh.faces_unique_edges.flatten(), return_counts=True
    )
    boundary_edges = mesh.edges_unique[
        mesh_edge_indexes[
            np.where(mesh_edge_counts == 1)[0]
        ]
    ]
    boundary_vertex_indexes = np.unique(boundary_edges.flatten())
    return boundary_vertex_indexes, boundary_edges


def remove_boundary_features(mesh, features, how='none'):
    """Removes features indexed into vertex edges adjacent to 1 face only.
    :param how: 'all_verts': remove entire feature curve if all vertices are boundary
                'edges': remove vertices that belong to boundary edges only (not to other edges)
                'verts': remove vertices that are boundary
                'none': do nothing
    """
    if how == 'none':
        return features

    boundary_vertex_indexes, boundary_edges = get_boundary_vertex_indexes(mesh)

    non_boundary_curves = []
    for curve in features['curves']:
        non_boundary_curve = deepcopy(curve)

        if how == 'all_verts':
            if np.all([vert_index in boundary_vertex_indexes
                       for vert_index in curve['vert_indices']]):
                continue

        elif how == 'verts':
            non_boundary_vert_indices = np.array([
                vert_index for vert_index in curve['vert_indices']
                if vert_index not in boundary_vertex_indexes
            ])
            if len(non_boundary_vert_indices) == 0:
                continue
            non_boundary_curve['vert_indices'] = non_boundary_vert_indices

        elif how == 'edges':
            curve_edges = mesh.edges_unique[
                np.where(
                    np.all(np.isin(mesh.edges_unique, curve['vert_indices']), axis=1)
                )[0]
            ]
            non_boundary = (curve_edges[:, None] != boundary_edges).any(2).all(1)
            non_boundary_vert_indices = np.unique(curve_edges[non_boundary])
#             if len(non_boundary_vert_indices) == 0:
#                 continue
            non_boundary_curve['vert_indices'] = non_boundary_vert_indices

        non_boundary_curves.append(non_boundary_curve)

    non_boundary_features = {
        'curves': non_boundary_curves,
        'surfaces': features.get('surfaces', [])
    }
    return non_boundary_features


In [ ]:
nbhood_features = compute_features_nbhood(
    mesh,
    features,
    item['orig_face_indexes'],
    item['orig_vert_indices'],
    deduce_verts_from_faces=False,
)

In [ ]:
len(nbhood_features['curves'])

In [ ]:
nbhood_features = remove_boundary_features(nbhood, nbhood_features, how='edges')

In [ ]:
len(nbhood_features['curves'])

In [ ]:
distances_a, directions_a, has_sharp_a = annotator.annotate(nbhood, nbhood_features, item['points'])

In [ ]:
projections, distances = annotator.do_annotate(nbhood, nbhood_features, item['points'])

In [ ]:
distances, directions = compute_bounded_labels(
    points, projections, distances=distances,
    max_distance=self.distance_upper_bound)

In [ ]:
display_sharpness(
    nbhood, plot_meshvert=True, meshvert_psize=HIGH_RES,
    sharp_vert=None, 
    shaarpvert_psize=HIGH_RES,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
    samples_psize=HIGH_RES,
    samples=item['points'],
    samples_distances=distances,
    directions=directions
)

In [9]:
from sharpf.data import DataGenerationException
from sharpf.utils.abc_utils.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.datasets.sharpf_io import save_point_patches
from sharpf.data.mesh_nbhoods import NBHOOD_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.data.point_samplers import SAMPLER_BY_TYPE
from sharpf.utils.abc_utils.abc.feature_utils import compute_features_nbhood, remove_boundary_features, get_curves_extents
from sharpf.utils.py_utils.console import eprint_t
from sharpf.utils.py_utils.os import add_suffix
from sharpf.utils.py_utils.config import load_func_from_config
from sharpf.utils.abc_utils.mesh.io import trimesh_load
from sharpf.utils.geometry import mean_mmd
import sharpf.data.data_smells as smells
from sharpf.utils.abc_utils.mesh.indexing import reindex_array, reindex_zerobased
from sharpf.utils.abc_utils.abc.feature_utils import get_surface_as_mesh

In [45]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

data_holders_by_chunk = {
    0: None,
    1: None,
}


def get_data_holder(chunk_id):
    global data_holders_by_chunk
    if None is data_holders_by_chunk.get(chunk_id):
        obj_filename = '/data/abc/abc_{}_obj_v00.7z'.format(str(chunk_id).zfill(4))
        feat_filename = '/data/abc/abc_{}_feat_v00.7z'.format(str(chunk_id).zfill(4))
        data_holders_by_chunk[chunk_id] = ABCChunk([obj_filename, feat_filename])
        
    data_holder = data_holders_by_chunk[chunk_id]
    return data_holder


def fix_patch(patch, chunk_ids, config):
    
    item = None
    for chunk_id in chunk_ids:
        try:
            data_holder = get_data_holder(chunk_id)
            item = data_holder.get(patch['item_id'].decode())
        except KeyError:
            continue
            
    if None is item:
        raise ValueError()

    mesh, _, _ = trimesh_load(item.obj)
    features = yaml.load(item.feat, Loader=yaml.Loader)


    shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
    base_n_points_per_short_curve = config.get('base_n_points_per_short_curve', 8)
    base_resolution_3d = config.get('base_resolution_3d', LOW_RES)
    short_curve_quantile = config.get('short_curve_quantile', 0.05)
    annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])

    mesh = scale_mesh(mesh, features, shape_fabrication_extent, base_resolution_3d,
                          short_curve_quantile=short_curve_quantile,
                          n_points_per_short_curve=base_n_points_per_short_curve)

    nbhood = reindex_zerobased(
        mesh,
        patch['orig_vert_indices'],
        patch['orig_face_indexes'])

    nbhood_features = compute_features_nbhood(
        mesh,
        features,
        patch['orig_face_indexes'],
        patch['orig_vert_indices'],
        deduce_verts_from_faces=False)

    nbhood_features = remove_boundary_features(
        nbhood,
        nbhood_features,
        how='edges')
    num_sharp_curves = len([curve for curve in nbhood_features['curves'] if curve['sharp']])

    distances, directions, has_sharp = annotator.annotate(
        nbhood,
        nbhood_features,
        patch['points'])
        
    fixed_part = {
        'distances': distances,
        'directions': directions,
        'has_sharp': has_sharp,
        'num_sharp_curves': num_sharp_curves,
    }
        
    return fixed_part

In [46]:
with open('/code/scripts/data_scripts/configs/pointcloud_datasets/dataset_config_high_res_clean.json') as config_file:
    config = json.load(config_file)

In [47]:
def scale_mesh(mesh, features, shape_fabrication_extent, resolution_3d,
               short_curve_quantile=0.05, n_points_per_short_curve=4):
    # compute standard size spatial extent
    mesh_extent = np.max(mesh.bounding_box.extents)
    mesh = mesh.apply_scale(shape_fabrication_extent / mesh_extent)

    # compute lengths of curves
    sharp_curves_lengths = get_curves_extents(mesh, features)

    least_len = np.quantile(sharp_curves_lengths, short_curve_quantile)
    least_len_mm = resolution_3d * n_points_per_short_curve

    mesh = mesh.apply_scale(least_len_mm / least_len)

    return mesh

In [59]:
%time fix = fix_patch(dataset[14], [0, 1], config)

CPU times: user 10.9 s, sys: 280 ms, total: 11.2 s
Wall time: 11.2 s


In [64]:
np.allclose(fix['directions'], dataset[14]['directions'])

True

In [62]:
dataset[14]

{'distances': array([0.25064018, 0.26161474, 0.2449492 , ..., 0.71209611, 0.02970788,
        0.10968887]),
 'normals': array([[ 0.        ,  0.        , -1.        ],
        [ 0.        ,  0.        , -1.        ],
        [ 0.        ,  0.        , -1.        ],
        ...,
        [ 0.        ,  0.        , -1.        ],
        [-0.44120994,  0.89737676,  0.00698177],
        [-0.26524471,  0.65872628, -0.70407737]]),
 'orig_face_indexes': array([ 1039,  1040,  1199,  1200,  1201,  1202,  1247,  1248,  1249,
         1250,  1285,  1286,  1287,  1288,  1381,  1382,  1383,  1384,
         1441,  1442,  1443,  1471,  1472,  1495,  1521,  1522,  1543,
         1544,  1546,  1567,  1568,  2565,  3184,  3329,  3375,  3422,
         3620,  8710,  8766,  8789,  9045,  9085,  9088,  9174,  9226,
         9234,  9293,  9305,  9329,  9353,  9393,  9418,  9448,  9489,
         9515,  9555,  9770,  9815, 10171, 10177, 10390, 10402, 10437,
        10476, 10590, 10616, 10674, 10887, 10954, 1096

In [40]:
display_sharpness(
    None, plot_meshvert=True, meshvert_psize=HIGH_RES,
    sharp_vert=None, 
    sharpvert_psize=HIGH_RES,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
    samples_psize=HIGH_RES,
    samples=dataset[14]['points'],
    samples_distances=fix['distances'],
    directions=fix['directions'] / np.linalg.norm()
)

Vectors(colors=array([], dtype=uint32), head_color=255, id=139759872265688, model_matrix=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32), origin_color=255, origins=array([[ 2.0992217 , -3.5293722 ,  0.        ],
       [ 2.1175919 , -3.531135  ,  0.        ],
       [ 2.1164966 , -3.5123491 ,  0.        ],
       ...,
       [ 1.8255069 , -4.1886873 ,  0.        ],
       [ 1.5249941 , -3.2285771 ,  0.31542143],
       [ 1.3850887 , -3.5156674 ,  0.07789262]], dtype=float32), type='Vectors', use_head=True, vectors=array([[-1.2912011e-01,  2.1482085e-01,  0.0000000e+00],
       [-1.3477376e-01,  2.2422709e-01,  0.0000000e+00],
       [-1.2618838e-01,  2.0994312e-01,  0.0000000e+00],
       ...,
       [-2.9084730e-01,  6.4999020e-01,  0.0000000e+00],
       [-9.1569105e-05,  1.8609840e-04, -2.9706152e-02],
       [ 2.8846428e-02, -7.1639121e-02, -7.7891916e-02]], dtype=float32))


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()